### Friends canned laughter identification

In this notebook, we will try to use data from available friends
episodes to try to train a model using VGGish

In [ ]:
import sys
sys.path.append('../utils/')

In [ ]:
# local imports
import utils
import episode
import color
import stats
# stdlib and package imports
import numpy as np
from pathlib import Path 
from matplotlib import pyplot as plt
from collections import Counter
# keras and ML imports
from keras.models import Sequential, Model, model_from_yaml
from keras.layers import Input, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize as sknormalize
from imblearn import under_sampling

#### Extracting data segments
Here we will extract labelled embeddings of wav data for each of the episodes

In [ ]:
# these are the episodes we have annotation data for
episodes = ['friends-s02-e{:0>2d}'.format(i) for i in range(1, 5)]

In [ ]:
# use VGGish to generate embeddings for each of the episode, and split data into chunks of 0.96s (no preserve_length)
# NOTE: run only if not run already
archive = Path('../data/friends-s02-e{01..04}_datachunks.npz')
if not archive.exists():
    X_raw, Y_raw, refs = episode.get_data(which_episodes=episodes, use_vggish=True, preserve_length=False)

In [ ]:
if not archive.exists():
    color.INFO('INFO', 'archive file doesn\'t exists; saving it')
    np.savez_compressed(archive, X_raw=X_raw, Y_raw=Y_raw, refs=refs)
color.INFO('INFO', 'loading data from numpy archive')
arrays = np.load(archive)
X_raw, Y_raw, refs = arrays['X_raw'], arrays['Y_raw'], arrays['refs']

In [ ]:
X_raw.shape, Y_raw.shape, refs.shape

In [ ]:
Y_raw[2310], refs[2310]

#### Now we'll use the extracted data to generate balanced training and testing data sets

First, resample data to have equal number of 'laugh' and 'no-laugh' examples

In [ ]:
rus = under_sampling.RandomUnderSampler(sampling_strategy='not minority')
X_res, Y_res = rus.fit_resample(X_raw, Y_raw)

In [ ]:
# did the resampling work?
Counter(Y_res)

Next, split data into training and testing sets so it doesn't get mixed up

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_res, Y_res, test_size=0.25, random_state=1,
                                                    stratify=Y_res.reshape(-1))

In [ ]:
# check again: did the split go as expected?
Counter(Y_train), Counter(Y_test)

#### Now we'll attempt to model the balanced data using a Keras dense model

In [ ]:
inp = Input(shape=(128,), name='in0')

layer = Dense(16, activation='relu', name='d0')(inp)
layer = Dropout(.4, name='dr0')(layer)

layer = Dense(8, activation='relu', name='d1')(layer)
layer = Dropout(.4, name='dr1')(layer)

layer = Dense(1, activation='sigmoid', name='out')(layer)

model = Model(inputs=[inp], outputs=[layer])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])

model.summary()

In [ ]:
# train model
H = model.fit(X_train, Y_train.reshape(-1), epochs=32, validation_data=[X_test, Y_test.reshape(-1)])

In [ ]:
# plot training history
stats.plot_history(H)

In [ ]:
# plot the ROC curve for this model and data
stats.plot_roc_curve(model=model, x=X_test, y_true=Y_test.reshape(-1))